<a href="https://colab.research.google.com/github/NayibGS07/Kaggle_Competitions/blob/main/Kaggle_Competition_Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [192]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
import keras_tuner as kt
from sklearn.preprocessing import StandardScaler


In [67]:
df_train = pd.read_csv('/content/train.csv')
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [68]:
df_train = df_train.drop(columns = ['PassengerId', 'Name'])
df_train

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [69]:
df_test = pd.read_csv('/content/test.csv')
df_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [70]:
df_test = df_test
df_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [71]:
df = pd.concat([df_train.drop(columns = ['Transported']), df_test.drop(columns = ['PassengerId', 'Name'])], ignore_index = True)
df

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
12965,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0
12966,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0
12967,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0
12968,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0


In [72]:
for col in ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']:
  le = preprocessing.LabelEncoder()
  le.fit(df[col])
  df_train[col+'_num'] = le.transform(df_train[col])
  df_test[col+'_num'] = le.transform(df_test[col])

In [74]:
X = df_train[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'HomePlanet_num', 'CryoSleep_num', 'Cabin_num', 'Destination_num', 'VIP_num']].fillna(-1)
y = df_train.Transported

In [75]:
# Implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66)

In [76]:
# Random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

# Predictions
rfc_predict = rfc.predict(X_test)

In [78]:
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')

In [79]:
print(f'''=== Confusion Matrix === \n 
          {confusion_matrix(y_test, rfc_predict)} \n
          === Classification Report === \n
          {classification_report(y_test, rfc_predict)} \n
          === All AUC Scores === \n
          {rfc_cv_score} \n
          === Mean AUC Score === \n
          Mean AUC Score - Random Forest: {rfc_cv_score.mean()}''')

=== Confusion Matrix ===
[[714 154]
 [214 657]]


=== Classification Report ===
              precision    recall  f1-score   support

       False       0.77      0.82      0.80       868
        True       0.81      0.75      0.78       871

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739



=== All AUC Scores ===
[0.87290187 0.83766172 0.84652461 0.83057621 0.82598631 0.86515908
 0.86893865 0.85775355 0.88639831 0.87376972]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8565670025083492


In [86]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features at every split
max_features = ['auto', 'sqrt']

# Max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)

# Create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }

# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the model
rfc_random.fit(X_train, y_train)

# Print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'n_estimators': 800, 'max_features': 'sqrt', 'max_depth': 220}


In [87]:
# Random forest model best creation
rfc_best = RandomForestClassifier(n_estimators = 800, max_features = 'sqrt', max_depth = 220)
rfc_best.fit(X_train, y_train)

# Predictions
rfc_best_predict = rfc_best.predict(X_test)

In [89]:
rfc_best_cv_score = cross_val_score(rfc_best, X, y, cv=10, scoring='roc_auc')

In [93]:
print(f'''=== Confusion Matrix ===
{confusion_matrix(y_test, rfc_best_predict)} \n \n
=== Classification Report ===
{classification_report(y_test, rfc_best_predict)} \n \n
 === All AUC Scores ===
{rfc_best_cv_score} \n \n
=== Mean AUC Score ===
Mean AUC Score - Random Forest: {rfc_best_cv_score.mean()}''')

=== Confusion Matrix ===
[[707 161]
 [207 664]] 
 

=== Classification Report ===
              precision    recall  f1-score   support

       False       0.77      0.81      0.79       868
        True       0.80      0.76      0.78       871

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739
 
 

 === All AUC Scores ===
[0.87859642 0.8395326  0.84347782 0.82945853 0.8342047  0.86983918
 0.8733221  0.86060611 0.88798483 0.87069203] 
 

=== Mean AUC Score ===
Mean AUC Score - Random Forest: 0.8587714321712798


In [94]:
prediction = rfc_best.predict(df_test[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'HomePlanet_num', 'CryoSleep_num', 'Cabin_num', 'Destination_num', 'VIP_num']].fillna(-1))

In [144]:
submission_rf_df = pd.DataFrame(df_test.PassengerId)
submission_rf_df['Transported'] = pd.Series(prediction)
submission_rf_df

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [145]:
submission_rf_df.to_csv('/content/submission_rf.csv')

In [216]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [218]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate = 0.00115),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
        tf.keras.metrics.Precision(name = 'precision'),
        tf.keras.metrics.Recall(name = 'recall')
    ]
)

earlystopping = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 100)

history = model.fit(X_train, y_train, epochs = 1000, validation_split = 0.1, callbacks = [earlystopping])

Epoch 1/1000
196/196 [==============================] - 2s 7ms/step - loss: 0.4727 - accuracy: 0.7726 - precision: 0.7712 - recall: 0.7878 - val_loss: 0.4744 - val_accuracy: 0.7629 - val_precision: 0.7295 - val_recall: 0.7595
Epoch 2/1000
196/196 [==============================] - 1s 4ms/step - loss: 0.4330 - accuracy: 0.7964 - precision: 0.7847 - recall: 0.8280 - val_loss: 0.4711 - val_accuracy: 0.7701 - val_precision: 0.7422 - val_recall: 0.7563
Epoch 3/1000
196/196 [==============================] - 1s 4ms/step - loss: 0.4253 - accuracy: 0.7929 - precision: 0.7850 - recall: 0.8179 - val_loss: 0.4726 - val_accuracy: 0.7672 - val_precision: 0.7484 - val_recall: 0.7342
Epoch 4/1000
196/196 [==============================] - 1s 4ms/step - loss: 0.4203 - accuracy: 0.7991 - precision: 0.7881 - recall: 0.8289 - val_loss: 0.4834 - val_accuracy: 0.7716 - val_precision: 0.7151 - val_recall: 0.8259
Epoch 5/1000
196/196 [==============================] - 1s 4ms/step - loss: 0.4133 - accuracy: 0

In [219]:
result = model.evaluate(X_test, y_test)
print("Accuracy: {}".format(result[1]))

55/55 [==============================] - 0s 7ms/step - loss: 1.3896 - accuracy: 0.7654 - precision: 0.7581 - recall: 0.7807
Accuracy: 0.7653824090957642


In [220]:
epochs = len(history.history['loss']) + 1
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history.history['loss'], mode = 'lines', name = 'Loss'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history.history['accuracy'], mode = 'lines', name = 'Accuracy'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history.history['precision'], mode = 'lines', name = 'Precision'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history.history['recall'], mode = 'lines', name = 'Recall'))
fig.update_layout(title='Evaluation metrics',
                   xaxis_title='Epoch', template = 'plotly_white')
fig.show()

In [221]:
epochs = len(history.history['val_loss']) + 1
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history.history['val_loss'], mode = 'lines', name = 'Val Loss'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history.history['val_accuracy'], mode = 'lines', name = 'Val Accuracy'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history.history['val_precision'], mode = 'lines', name = 'Val Precision'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history.history['val_recall'], mode = 'lines', name = 'Val Recall'))
fig.update_layout(title='Evaluation metrics',
                   xaxis_title='Epoch', template = 'plotly_white')
fig.show()

In [222]:
predictions = model.predict(X_test)
prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(predictions)]

55/55 [==============================] - 0s 1ms/step


In [223]:
print(f'''=== Confusion Matrix ===
{confusion_matrix(y_test, prediction_classes)} \n \n
=== Classification Report ===
{classification_report(y_test, prediction_classes)}''')

=== Confusion Matrix ===
[[651 217]
 [191 680]] 
 

=== Classification Report ===
              precision    recall  f1-score   support

       False       0.77      0.75      0.76       868
        True       0.76      0.78      0.77       871

    accuracy                           0.77      1739
   macro avg       0.77      0.77      0.77      1739
weighted avg       0.77      0.77      0.77      1739



In [224]:
prediction = model.predict(scaler.fit_transform(df_test[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'HomePlanet_num', 'CryoSleep_num', 'Cabin_num', 'Destination_num', 'VIP_num']].fillna(-1)))
prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(prediction)]

134/134 [==============================] - 0s 1ms/step


In [225]:
submission_model_df = pd.DataFrame(df_test.PassengerId)
submission_model_df['Transported'] = pd.Series(prediction_classes)
submission_model_df['Transported'] = submission_model_df.Transported.replace([0, 1], [False, True])
submission_model_df

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [226]:
submission_model_df.to_csv('/content/submission_model.csv')

In [239]:
def build_model(hp):
    # Initialize sequential API and start building model.
    model = tf.keras.Sequential()

    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 5
    # Number of Units: 32 - 512 with stepsize of 32
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
            tf.keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation="relu")
            )
        
        # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
        model.add(tf.keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(
      loss=tf.keras.losses.binary_crossentropy,
      optimizer=tf.keras.optimizers.legacy.Adam(learning_rate =hp_learning_rate),
      metrics=[
          tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
          tf.keras.metrics.Precision(name = 'precision'),
          tf.keras.metrics.Recall(name = 'recall')
        ]
    )

    return model

In [240]:
tuner = kt.Hyperband(build_model,
                     objective = "val_accuracy",
                     max_epochs = 100,
                     factor = 3,
                     hyperband_iterations = 10,
                     directory = "kt_dir",
                     project_name = "kt_hyperband",)

In [241]:
earlystopping = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 100)

tuner.search(X_train, y_train, epochs = 1000, validation_split=0.1, callbacks=[earlystopping])


In [242]:
best_hps=tuner.get_best_hyperparameters()[0]

h_model = tuner.hypermodel.build(best_hps)

earlystopping = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 100)

history_best = h_model.fit(X_train, y_train, epochs = 1000, validation_split = 0.1, callbacks = [earlystopping])

Epoch 1/1000
196/196 [==============================] - 2s 6ms/step - loss: 29.1699 - accuracy: 0.5981 - precision: 0.6008 - recall: 0.6315 - val_loss: 0.6464 - val_accuracy: 0.5848 - val_precision: 0.5443 - val_recall: 0.5253
Epoch 2/1000
196/196 [==============================] - 1s 4ms/step - loss: 0.6806 - accuracy: 0.6360 - precision: 0.6152 - recall: 0.7637 - val_loss: 0.5978 - val_accuracy: 0.6437 - val_precision: 0.6250 - val_recall: 0.5380
Epoch 3/1000
196/196 [==============================] - 1s 4ms/step - loss: 0.6296 - accuracy: 0.6735 - precision: 0.6404 - recall: 0.8204 - val_loss: 0.6861 - val_accuracy: 0.5489 - val_precision: 0.5016 - val_recall: 0.9842
Epoch 4/1000
196/196 [==============================] - 1s 4ms/step - loss: 0.5746 - accuracy: 0.7224 - precision: 0.6978 - recall: 0.8038 - val_loss: 0.6126 - val_accuracy: 0.6480 - val_precision: 0.5664 - val_recall: 0.9589
Epoch 5/1000
196/196 [==============================] - 1s 4ms/step - loss: 0.5427 - accuracy: 

In [243]:
result = h_model.evaluate(X_test, y_test)
print("Accuracy: {}".format(result[1]))

55/55 [==============================] - 0s 3ms/step - loss: 0.6932 - accuracy: 0.5009 - precision: 0.5009 - recall: 1.0000
Accuracy: 0.5008625388145447


In [244]:
epochs = len(history_best.history['loss']) + 1
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history_best.history['loss'], mode = 'lines', name = 'Loss'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history_best.history['accuracy'], mode = 'lines', name = 'Accuracy'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history_best.history['precision'], mode = 'lines', name = 'Precision'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history_best.history['recall'], mode = 'lines', name = 'Recall'))
fig.update_layout(title='Evaluation metrics',
                   xaxis_title='Epoch', template = 'plotly_white')
fig.show()

In [245]:
epochs = len(history_best.history['val_loss']) + 1
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history_best.history['val_loss'], mode = 'lines', name = 'Val Loss'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history_best.history['val_accuracy'], mode = 'lines', name = 'Val Accuracy'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history_best.history['val_precision'], mode = 'lines', name = 'Val Precision'))
fig.add_trace(go.Scatter(x = np.arange(1, epochs), y = history_best.history['val_recall'], mode = 'lines', name = 'Val Recall'))
fig.update_layout(title='Evaluation metrics',
                   xaxis_title='Epoch', template = 'plotly_white')
fig.show()

In [246]:
predictions = h_model.predict(X_test)
prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(predictions)]

55/55 [==============================] - 0s 2ms/step


In [247]:
print(f'''=== Confusion Matrix ===
{confusion_matrix(y_test, prediction_classes)} \n \n
=== Classification Report ===
{classification_report(y_test, prediction_classes)}''')

=== Confusion Matrix ===
[[  0 868]
 [  0 871]] 
 

=== Classification Report ===
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       868
        True       0.50      1.00      0.67       871

    accuracy                           0.50      1739
   macro avg       0.25      0.50      0.33      1739
weighted avg       0.25      0.50      0.33      1739



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [248]:
prediction = h_model.predict(scaler.fit_transform(df_test[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'HomePlanet_num', 'CryoSleep_num', 'Cabin_num', 'Destination_num', 'VIP_num']].fillna(-1)))
prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(prediction)]

134/134 [==============================] - 0s 1ms/step


In [249]:
submission_model_df = pd.DataFrame(df_test.PassengerId)
submission_model_df['Transported'] = pd.Series(prediction_classes)
submission_model_df['Transported'] = submission_model_df.Transported.replace([0, 1], [False, True])
submission_model_df

,PassengerId,Transported
0,0013_01,True
1,0018_01,True
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [250]:
submission_model_df.to_csv('/content/submission_model.csv')